In [ ]:
# https://www.tpex.org.tw/zh-tw/

In [1]:
import requests # type: ignore
import pandas as pd # type: ignore
import psycopg2 # type: ignore
from psycopg2 import sql # type: ignore
from tqdm import tqdm # type: ignore
from datetime import datetime, timedelta
import random
import time
from util import get_init_period_trade_cal, db_params, \
    create_table, get_all_codes, sleeper, \
    get_ts, init_trade_cal, get_trade_cal_from_postgres, get_latest_da

In [5]:
import sys, ssl, requests, urllib3
try:
    import certifi
    cafile = certifi.where()
except Exception:
    cafile = None

print("PY:", sys.executable)
print("VER:", sys.version)
print("SSL:", ssl.OPENSSL_VERSION)
print("requests:", requests.__version__, "urllib3:", urllib3.__version__)
print("CA file:", cafile)

PY: /Applications/Xcode.app/Contents/Developer/usr/bin/python3
VER: 3.9.6 (default, Mar 12 2025, 20:22:46) 
[Clang 17.0.0 (clang-1700.0.13.3)]
SSL: LibreSSL 2.8.3
requests: 2.32.3 urllib3: 1.26.20
CA file: /Users/tp_mini/Library/Python/3.9/lib/python/site-packages/certifi/cacert.pem


In [ ]:
# create tables, trade_cal
for i in ['tpex_peratio', 'turnover']:
    create_table(i) 

create tpex_peratio


In [ ]:
def get_tpex_maincode():
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    # tpex maincode
    url = "https://www.tpex.org.tw/www/zh-tw/afterTrading/dailyQuotes?date=2025%2F09%2F04&response=json"
    res = requests.get(url)
    data = res.json()
    data_copy = data.copy()
    data_copy = data_copy['tables'][0]['data']

    data_filtered = []
    for i in range(len(data_copy)):
        if "購" in data_copy[i][1]:
            continue
        if "售0" in data_copy[i][1]:
            continue
        if "U" in data_copy[i][0]:
            continue
        if data_copy[i][2] == "---":
            print(data_copy[i])
            continue
        if data_copy[i][3] == "---":
            continue
        if data_copy[i][4] == "---":
            continue
        if data_copy[i][5] == "---":
            continue
        if data_copy[i][6] == "---":
            continue
        a = data_copy[i][0:3]
        
        b = data_copy[i][4:7]
        c = data_copy[i][8:11]
        d = data_copy[i][15]
        data_filtered.append(a + b + c + [d])

    for i in tqdm(data_filtered):
        sql_str = f"""
        INSERT INTO public.maincode (code, cname, exchange)
        VALUES ('{i[0]}', '{i[1]}', 'tpex') on conflict (code) do nothing;
        """
        cursor.execute(sql_str)
        conn.commit()

def get_tpex_maincode_v2():
    '''
    from pe_ratio also but from postgres
    '''
    # tpex stock list from postgres
    sql_str = ""
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(f"""
    select distinct code, cname from public.tpex_peratio;
                    """)
    codes = cursor.fetchall()
    codes = [i[0] for i in codes]
    for code in tqdm(codes):
        sql_str += f"""INSERT INTO public.maincode (code, cname, exchange)
        VALUES ('{code[0]}', '{code[1]}', 'tpex') on conflict (code) do nothing;"""
    cursor.execute(sql_str)
    conn.commit()
    return None
# get_tpex_maincode()
# get_tpex_maincode_v2()

In [ ]:
def get_tpex_peratio():
    # tpex pe_ratio

    '''
    uri = "https://www.tpex.org.tw/www/zh-tw/afterTrading/PERatio"
    uri = "https://www.tpex.org.tw/www/zh-tw/afterTrading/PERatio?date=2025%2F09%2F03&response=json"
    date=2025%2F09%2F04&id=&response=json
    '''

    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    trade_cal = get_init_period_trade_cal('tpex_peratio', t_='postgres')

    if len(trade_cal) > 0:
        for dat in tqdm(trade_cal):
            sleeper(50)
            year,month,day = dat.strftime('%Y-%m-%d').split('-')
            uri = f"https://www.tpex.org.tw/www/zh-tw/afterTrading/PERatio?date={year}%2F{month}%2F{day}&response=json"
            
            res = requests.get(uri)
            data = res.json()['tables'][0]['data']
            sql_str = ""
            da = f"{year}{month}{day}"
            for i in data:
                i[5] = i[5].replace('N/A', "0")
                i[3] = i[3].replace(',', '')
                i[4] = i[4].replace(',', '')
                i[5] = i[5].replace(',', '')
                sql_str += f"""INSERT INTO public.tpex_peratio (da, code, cname, cl, eps, pe_ratio)
                VALUES ('{da}', '{i[1]}', '{i[2]}', '{i[3]}', '{i[4]}', '{i[5]}') on conflict (da, code) do nothing;"""
            cursor.execute(sql_str)
            conn.commit()
    else:
        print("Error: get_tpex_peratio No new data")
get_tpex_peratio()

No new data


In [ ]:
def init_tpex_price():
    '''
    url = "https://www.tpex.org.tw/www/zh-tw/afterTrading/dailyQuotes?date=2025%2F09%2F04&response=json"
    https://www.tpex.org.tw/zh-tw/mainboard/trading/info/pricing.html
    https://www.tpex.org.tw/www/zh-tw/afterTrading/dailyQuotes?date=2025%2F09%2F04&response=json
    '''
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    trade_cal = get_trade_cal_from_postgres()
    for da in tqdm(trade_cal, desc="tpex price"):
        year, month, day = da.strftime('%Y-%m-%d').split('-')
        url = f"https://www.tpex.org.tw/www/zh-tw/afterTrading/dailyQuotes?date={year}%2F{month}%2F{day}&response=json"

        res = requests.get(url)
        data = res.json()['tables'][0]['data']
        data_copy = data.copy()
        da = res.json()['date']

        data_filtered = []
        for i in range(len(data_copy)):
            if "購" in data_copy[i][1]:
                continue
            if "售0" in data_copy[i][1]:
                continue
            if "U" in data_copy[i][0]:
                continue
            if data_copy[i][2] == "---":
                print(data_copy[i])
                continue
            if data_copy[i][3] == "---":
                continue
            if data_copy[i][4] == "---":
                continue
            if data_copy[i][5] == "---":
                continue
            if data_copy[i][6] == "---":
                continue
            a = data_copy[i][0]
            b = data_copy[i][2]
            e = data_copy[i][4:7]
            c = data_copy[i][8:11]
            d = data_copy[i][15]
            data_filtered.append([a] + [b] + e + c + [d])

        for i in data_filtered:
            for j in [2, 3, 4, 5, 6, 7, -1, -2]:
                if isinstance(i[j], str):
                    i[j] = i[j].replace(',', '')
            sql_str = f"""
            INSERT INTO public.price (da, code, cl, op, hi, lo, stock_volume, cash_volume, number_of_trades)
            VALUES ('{da}', '{i[0]}', '{i[1]}', '{i[2]}', '{i[3]}', '{i[4]}', '{i[5]}', '{i[6]}', '{i[7]}') on conflict (da, code) do nothing;
            """
            cursor.execute(sql_str)
            conn.commit()

            # # shares issued
            # sql_str = f"""
            # INSERT INTO public.shares_issued (code, shares_issued)
            # VALUES ('{i[0]}', {i[2]}) on conflict (code) do nothing;
            # """
            # cursor.execute(sql_str)
            # conn.commit()

            # # shares_issued_change

In [ ]:
# tpex volume
'''
https://www.tpex.org.tw/zh-tw/mainboard/trading/realtime/amount.html
'''

In [ ]:
# tpex indices
'''
https://www.tpex.org.tw/zh-tw/mainboard/trading/info/indices-pricing.html
https://www.tpex.org.tw/www/zh-tw/afterTrading/indexSummary?date=2025%2F09%2F04&response=json
'''

In [ ]:
# tpex highlight
'''
https://www.tpex.org.tw/zh-tw/mainboard/trading/info/highlight.html
'''

In [ ]:
# tpex short list
'''
https://www.tpex.org.tw/zh-tw/mainboard/trading/info/short.html
'''

In [ ]:
# tpex pe_ratio (like twse)
'''
https://www.tpex.org.tw/zh-tw/mainboard/trading/info/daily-pe.html
'''

In [ ]:
# tpex market cap
'''
https://www.tpex.org.tw/zh-tw/mainboard/trading/historical/market-value.html
'''

# tpex turnover
'''
https://www.tpex.org.tw/zh-tw/mainboard/trading/historical/turnover-ratio/day.html
'''

# 
'''
https://www.tpex.org.tw/zh-tw/mainboard/trading/historical/rank-volume/day.html
https://www.tpex.org.tw/zh-tw/mainboard/trading/historical/rank-amount/day.html
https://www.tpex.org.tw/zh-tw/mainboard/trading/historical/avg-volume/day.html
https://www.tpex.org.tw/zh-tw/mainboard/trading/historical/avg-amount/day.html
'''